In [1]:

import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import time
import os
import json
import importlib
from matplotlib import pyplot as plt
import preprocess
# import preprocess as prep

import SLR_model 
import SLR_model_CNN_GRU
# import SLR_model_CL_GRU
import numpy as np



# model will output multiple(5) results/sec, how are we gonna handle it?


2024-11-19 19:22:31.034582: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 19:22:31.040449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-19 19:22:31.047723: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-19 19:22:31.049839: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 19:22:31.055381: I tensorflow/core/platform/cpu_feature_guar

2.17.0


I0000 00:00:1732011751.963309  118377 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732011751.983985  118377 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732011751.984892  118377 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732011751.987149  118377 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
load_size = 1500 # number of data to be loaded at once
epochs = 30
run_time= 5
batch_size = 8
save_dir = "saves"
load_dir = "saves"
model = SLR_model.get_model()

In [3]:
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)

In [ ]:
end_file=preprocess.getoutputdir()

save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"+"_final"

check_path = os.path.join(save_dir,'ckpt',ckpt_name)
hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
filepath=f'{check_path}.keras',
monitor='categorical_accuracy',
mode='max',
save_freq='epoch',
save_best_only=True)


start_person=1
start_count=1
current_word=" "
for i in sorted(os.listdir(preprocess.getoutputdir())):
    l_raws, r_raws, p_raws, y_raws, loss_weights_raws = [], [], [], [], []
    if start_person > int(i):
        continue

    video_list = sorted(os.listdir(os.path.join(preprocess.getoutputdir(), i)))
    batch_start_index = 0

    for k in range(1, run_time + 1):
        if start_count > k:
            continue
        elif start_count == k:
            start_count = 0

        for j_idx, j in enumerate(video_list):
            if len(j)!=8:
                continue
            if len(l_raws) == 0:
                current_word = j  # 배치 시작 단어 설정

            # 데이터 로드
            l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
            l_raws.append(l_raw)
            r_raws.append(r_raw)
            p_raws.append(p_raw)
            y_raws.append(y_raw)
            loss_weights_raws.append(loss_weights_raw)

            # 배치 처리
            if len(l_raws) >= load_size or j_idx == len(video_list) - 1:  # 배치 크기 도달 또는 마지막 파일
                end_word = j  # 배치 마지막 단어 설정

                # 로그 작성
                with open(os.path.join('logs', f"{ckpt_name}.txt"), 'a') as logs:
                    logs.write(
                        f'{time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {current_word} ~ {end_word}\n'
                    )
                print(
                    f'{time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {current_word} ~ {end_word}'
                )

                # 데이터 직렬화 및 모델 학습
                l_train, window_count = SLR_model.serialize(l_raws)
                r_train, window_count = SLR_model.serialize(r_raws)
                p_train, window_count, sample_weights = SLR_model.serialize(
                    p_raws, stride=2, loss_weights_list=loss_weights_raws, is_pose=True
                )
                x_train = (l_train, r_train, p_train)
                y_train = np.repeat(y_raws, window_count)
                y_train = SLR_model.encode_onehot2d(y_train)
                dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)

                hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])

                # 학습 기록 저장
                with open(hist_path, 'w') as file:
                    json.dump(hist.history, file)

                # 데이터 초기화
                l_raws.clear()
                r_raws.clear()
                p_raws.clear()
                y_raws.clear()
                loss_weights_raws.clear()
    

19-23-42:1) person:1 : 0001.npz ~ 1592.npz
Epoch 1/30


I0000 00:00:1732011825.236687  118486 service.cc:146] XLA service 0x712b240096b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732011825.236709  118486 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2024-11-19 19:23:45.283680: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-19 19:23:45.500328: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101


  18/2010 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - categorical_accuracy: 0.0982 - loss: 5.4248  

I0000 00:00:1732011828.436903  118486 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2010/2010 ━━━━━━━━━━━━━━━━━━━━ 25s 10ms/step - categorical_accuracy: 0.0027 - loss: 5.2577
Epoch 2/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - categorical_accuracy: 0.0013 - loss: 5.0253
Epoch 3/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0056 - loss: 4.8263
Epoch 4/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0103 - loss: 4.5892
Epoch 5/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0284 - loss: 4.2835
Epoch 6/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0424 - loss: 4.0806
Epoch 7/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0593 - loss: 3.8648
Epoch 8/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.0791 - loss: 3.7195
Epoch 9/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.1153 - loss: 3.4887
Epoch 10/30
2010/2010 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - categorical_accuracy: 0.1326 - loss

In [ ]:
# LEGACY TRAINING LOOP WITH LOAD SIZE  

#  # reload model file
# end_file=preprocess.getoutputdir()

# save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
# ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

# check_path = os.path.join(save_dir,'ckpt',ckpt_name)
# hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
# filepath=f'{check_path}.keras',
# monitor='categorical_accuracy',
# mode='max',
# save_freq='epoch',
# save_best_only=True)


# start_person=1
# start_count=1
# current_word=" "

# for i in range(1,17):
#     l_raws=[]
#     r_raws=[]
#     p_raws=[]
#     y_raws=[]
#     loss_weights_raws=[]
#     if start_person>i:
#         continue
#     for k in range(1,run_time+1):
#         if start_count>k:
#             continue
#         elif start_count==k:
#             start_count=0
#         for j in sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i)))):
#             if len(l_raws)==0:
#                 current_word = j
#             else:
#                 end_word = j
#             l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
#             l_raws.append(l_raw)
#             r_raws.append(r_raw)
#             p_raws.append(p_raw)
#             y_raws.append(y_raw)
#             loss_weights_raws.append(loss_weights_raw)
#             # print(p_raw.shape)
#             # break
#             if len(l_raws)>=load_size:
#                 with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
#                     logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {current_word} ~ {end_word}\n')  # 한 줄 쓰기
#                     l_train, window_count = SLR_model.serialize(l_raws)
#                     r_train, window_count = SLR_model.serialize(r_raws)
#                     p_train, window_count, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws, is_pose = True)
#                     x_train = (l_train, r_train, p_train)
                    
#                     y_train = np.repeat(y_raws, window_count)
#                     y_train = SLR_model.encode_onehot2d(y_train)
                    
#                     dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
#                     hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])

#                     with open(hist_path, 'w') as file:
#                         json.dump(hist.history, file)
#                     l_raws.clear()
#                     r_raws.clear()
#                     p_raws.clear()
#                     y_raws.clear()
#                     loss_weights_raws.clear()
#             if len(l_raws)>0:
#                 l_train, window_count = SLR_model.serialize(l_raws)
#                 r_train, window_count = SLR_model.serialize(r_raws)
#                 p_train, window_count, sample_weights = SLR_model.serialize(p_raws, stride=2, loss_weights_list=loss_weights_raws, is_pose = True)
#                 x_train = (l_train, r_train, p_train)
                
#                 y_train = np.repeat(y_raws, window_count)
#                 y_train = SLR_model.encode_onehot2d(y_train)
                
                
#                 dataset = SLR_model.convert_to_dataset(x_train, y_train, batch_size, sample_weights)
#                 hist = model.fit(dataset, epochs=epochs, callbacks=[model_checkpoint_callback])

#                 with open(hist_path, 'w') as file:
#                     json.dump(hist.history, file)



In [ ]:
# importlib.reload(SLR_model_CNN_GRU)

In [ ]:
# cngru_model = SLR_model_CNN_GRU.reinit_model(run_eagerly=True)

In [ ]:
# # PER VIDEO TRAINING LOOP

# end_file=preprocess.getoutputdir()

# save_suffix = time.strftime("%m-%d-%H", time.localtime(time.time()))
# ckpt_name=save_suffix+"-"+str(epochs)+"epochs-"+str(run_time)+"times"

# check_path = os.path.join(save_dir,'ckpt',ckpt_name)
# hist_path = os.path.join(save_dir, "hist",ckpt_name+".json")

# model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
# filepath=f'{check_path}.keras',
# monitor='categorical_accuracy',
# mode='max',
# save_freq='epoch',
# save_best_only=True)


# start_person=1

# for i in range(1,17):
#     if start_person>i:
#         continue
#     for k in range(1,run_time+1):
#         chkpt_elapsed = 0
#         file_list = sorted(os.listdir(os.path.join(preprocess.getoutputdir(),str(i))))
#         for j in file_list: # every video
#             print(j)
#             l_raw, r_raw, p_raw, y_raw, loss_weights_raw = preprocess.load_data(f"{i}/{j}")
#             with open(os.path.join('logs',ckpt_name+'.txt'), 'a') as logs:
#                 hist = None # declaration for scope
#                 logs.write(f'{ time.strftime("%H-%M-%S", time.localtime(time.time()))}:{k}) person:{i} : {j}\n')  # 한 줄 쓰기
#                 ## ----------------------------------------------------
#                 ## CLGRU <-> CNNGRU 전환시 serialize 인자값 바꿔줄 것
#                 ## --------------------------------------
#                 # PER VIDEO TRAINING LOOP
#                 l_train, window_count = SLR_model_CNN_GRU.serialize(l_raw, stride=1, window_size=17, hop_length=10)
#                 r_train, window_count = SLR_model_CNN_GRU.serialize(r_raw, stride=1, window_size=17, hop_length=10)
#                 p_train, window_count, sample_weights = SLR_model_CNN_GRU.serialize(p_raw, stride=2, window_size=17, hop_length=10, loss_weights=loss_weights_raw, is_pose=True)
#                 x_train = (l_train, r_train, p_train)

#                 # y_train = np.repeat(y_raw, window_count)
#                 # print(y_raw.shape)
#                 y_train = SLR_model_CNN_GRU.encode_onehot2d(y_raw)
#                 y_train = tf.expand_dims(y_train, axis=0)
#                 # print('yt:', y_train.shape)
#                 # print("li:",x_train[0].shape)
#                 # print("ri:",x_train[1].shape)
#                 # print(x_train[2].shape)
#                 # print(sample_weights.shape)
#                 # dataset = SLR_model_CNN_GRU.convert_to_dataset(x_train, y_train, batch_size= 1)#, sample_weights= sample_weights)
#                 # hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs)
                                
#                 if chkpt_elapsed < load_size and j is not file_list[-1]:
#                     # train without checkpoint
#                     # print(cngru_model.summary())
#                     hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs)
#                     chkpt_elapsed += 1
#                 else:
#                     # train with checkpoint every {load_size}th video
#                     hist = cngru_model.fit(x_train, y_train, batch_size=1, epochs=epochs, callbacks=[model_checkpoint_callback])
#                     chkpt_elapsed = 0   

#                 with open(hist_path, 'w') as file:
#                     json.dump(hist.history, file)

In [ ]:
# # Configurations
# load_size = 3000 # number of data to be loaded at once
# epochs = 50
# run_time = 2
# batch_size = 16
# save_dir = "saves"
# load_dir = "saves"
# # load_path = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints.keras"


In [ ]:
# model = SLR_model.get_model()

In [ ]:
# # 세이브된 모델 로드
# load_path = os.join(load_dir, "check_00-00-00.keras")
# model = SLR_model.load_model(load_path)

In [ ]:
# # 안될때 (강제중지 + 초기화)
# tf.keras.backend.clear_session()

In [ ]:
# with open(os.path.join("saves","hist","11-17-21-20epochs-3times.json"), 'r') as f:
#     hist = json.load(f)

#     plt.plot(range(len(hist['loss'])), hist['loss'])
#     plt.scatter(range(len(hist['loss'])), hist['loss'])
#     plt.xlabel("epochs")
#     plt.ylabel("loss")
#     plt.legend(["loss"])
#     plt.ylim((0,1))
#     plt.show()

In [ ]:
# print(os.listdir(os.path.join(preprocess.getoutputdir(),str(15))))

In [ ]:
# model.metrics_names